## Introduction

In [1]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.metrics import accuracy_score, f1_score

np.random.seed(1234)

C:\Users\mique\AppData\Local\Temp\ipykernel_11416\3232202249.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# MODEL CLASSIFICATION

First of all, read the preprocessed data. It is important to be this data and not the original, because they are saved in a .csv file.

In [2]:
# read clean data
data = pd.read_csv('../data/clean_data.csv')

### Resampling protocl

It is very important to make a good resampling protocol.

For further information, check out the report.

In [3]:
X = data.loc[:, data.columns != "Severity"]
y = data["Severity"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)

### DECISION TREE CLASSIFIER

In [5]:
# X = data.drop('Severity', axis=1)
# y = data['Severity']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)

# model = DecisionTreeClassifier(random_state=1234)

# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234, )

# param_grid = {
#     'max_depth': [3, 10, 30, None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

# grid_search = GridSearchCV(model, param_grid = param_grid, cv = cv, scoring='f1_weighted', n_jobs=-1)

# grid_search.fit(X_train, y_train)

# best_params = grid_search.best_params_
# print(best_params)

# best_model = DecisionTreeClassifier(**best_params)
# best_model.fit(X_train, y_train)

# y_pred = best_model.predict(X_test)

In [6]:
# print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
# print(f'F1 Score weightened: {f1_score(y_test, y_pred, average="weighted")}')
# print(f'F1 Score macro: {f1_score(y_test, y_pred, average="macro")}')
# print(f'F1 Score micro: {f1_score(y_test, y_pred, average="micro")}')

In [7]:
# from sklearn.metrics import roc_curve, auc
# from sklearn.preprocessing import label_binarize
# import matplotlib.pyplot as plt

# y_test_binarized = label_binarize(y_test, classes=[1, 2, 3, 4])
# y_pred_proba = best_model.predict_proba(X_test)


# fpr = dict()
# tpr = dict()
# roc_auc = dict()
# for i in range(4): 
#     fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_pred_proba[:, i])
#     roc_auc[i] = auc(fpr[i], tpr[i])

# plt.figure()
# colors = ['blue', 'red', 'green', 'orange'] 
# for i, color in zip(range(4), colors):
#     plt.plot(fpr[i], tpr[i], color=color, lw=2,
#              label='ROC curve of class {0} (area = {1:0.2f})'
#              ''.format(i+1, roc_auc[i]))

# plt.plot([0, 1], [0, 1], 'k--', lw=2)
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic (ROC) Curve')
# plt.legend(loc="lower right")
# plt.show()


## LOGISTIC REGRESSION

In [13]:
logreg = LogisticRegressionCV(Cs=5, cv = 5, scoring = 'f1_weighted', multi_class='multinomial', random_state = 1234)

logreg.fit(X_train, y_train)

/home/pol/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/pol/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n

LogisticRegressionCV(Cs=5, cv=5, multi_class='multinomial', random_state=1234,
                     scoring='f1_weighted')

In [9]:
optimal_C = logreg.Cs_[logreg.scores_[1].mean(axis=0).argmax()]
print("Optimal value for C:", optimal_C)

Optimal value for C: 10000.0


In [12]:
logreg = LogisticRegression(C=optimal_C, multi_class='multinomial')
cross_val_results = pd.DataFrame(cross_validate(logreg, X_train, y_train, cv = 5, scoring = ['accuracy', 'f1_macro', 'precision_macro', 'recall_macro'] ))

print(cross_val_results)

/home/pol/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/pol/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/pol/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHE

   fit_time  score_time  test_accuracy  test_f1_macro  test_precision_macro  \
0  4.472640    0.042470       0.802048       0.222538              0.200512   
1  4.761011    0.039888       0.802048       0.222538              0.200512   
2  4.389025    0.038607       0.801857       0.223758              0.294337   
3  4.285354    0.042983       0.802048       0.222538              0.200512   
4  4.605562    0.044959       0.802067       0.222541              0.200517   

   test_recall_macro  
0           0.250000  
1           0.250000  
2           0.250436  
3           0.250000  
4           0.250000  


/home/pol/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/pol/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
logreg = LogisticRegression(C=optimal_C, multi_class='multinomial')
cross_val_results = pd.DataFrame(cross_validate(logreg, X_train, y_train, cv = 5, scoring = ['accuracy', 'f1_macro', 'precision_macro', 'recall_macro'] ))

/home/pol/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/pol/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/pol/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHE

In [15]:
print(cross_val_results)

   fit_time  score_time  test_accuracy  test_f1_macro  test_precision_macro  \
0  4.023393    0.037168       0.802048       0.222538              0.200512   
1  4.329232    0.035230       0.802048       0.222538              0.200512   
2  4.226688    0.039190       0.801857       0.223758              0.294337   
3  4.219748    0.042141       0.802048       0.222538              0.200512   
4  4.211854    0.041782       0.802067       0.222541              0.200517   

   test_recall_macro  
0           0.250000  
1           0.250000  
2           0.250436  
3           0.250000  
4           0.250000  


## QDA

This model is used to predict the class from an input feature space. The model is based on prior probabilities on the input data. It assumes they are Gaussian distributed, so it was important to normalize the data in the preprocessing.

In [ ]:
#We will do a 5-fold cross-validation to find the best hyperparameters for the QDA model
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234, )

param_grid = {'reg_param': [0, 0.1, 0.2, 0.5]}
qda_model = QuadraticDiscriminantAnalysis()
grid_search = GridSearchCV(qda_model, param_grid = param_grid, cv = cv, scoring='f1_weighted', n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_score = grid_search.best_score_

results_df = pd.DataFrame({'Best_Parameters': [best_params], 'Best_F1_Weighted': [best_score]}, index=['QDA'])

print(results_df)

best_qda_model = QuadraticDiscriminantAnalysis(**best_params)
best_qda_model.fit(X_train, y_train)

y_pred = best_qda_model.predict(X_test)
f1 = f1_score(y_test, y_pred, average='weighted')

results_df = pd.DataFrame({'F1_Weighted': [f1]}, index=['QDA_test'])

print(results_df)